In [129]:
import pandas as pd
import numpy as np
from datetime import datetime, date
from sweref99 import projections
import folium
from folium import plugins
from selenium import webdriver
import matplotlib.pyplot as plt
from sklearn import linear_model, datasets
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display, clear_output

In [130]:
path='./data/Insatser till brand i skog och mark 2000-2020.xlsx'

In [131]:
#tm is used for the conversions of easting and northing to longitude and latitude
tm = projections.make_transverse_mercator("SWEREF_99_TM")

In [132]:
df = pd.read_excel(path)


In [133]:
df['TotArea'] = df['arealProduktivSkogsmark_m2']+df['arealAnnanTradbevuxenMark_m2']+df['arealMarkUtanTrad_m2']
df['Acres'] = df['TotArea']/4046.86

In [134]:
copy_df = df
copy_df = copy_df[copy_df.Acres>500]

In [136]:
sample = copy_df.sample(50, replace=True)

In [137]:
for column in df:
    if df[column].isnull().any():
       print('{0} has {1} null values'.format(column, df[column].isnull().sum()))

sweref99Norr has 20718 null values
sweref99Ost has 20717 null values


In [138]:
#Function that calculates number of missing data in column of dataframe and prints result.
def missing(df,column):
       x = len(df)
       if df[column].isnull().any():
           print('{0} has total of {1} null values'.format(column, df[column].isnull().sum()))
           print ('In the column {0}'.format(column), round(df[column].count()-1/x * 100, 3), '% of the cells have missing values')
 

In [139]:
#Missing values of column 'Gras'
missing(sample,'sweref99Norr')
missing(sample,'sweref99Ost')
sample.isnull().sum(axis = 0)

sweref99Norr has total of 1 null values
In the column sweref99Norr 47.0 % of the cells have missing values
sweref99Ost has total of 1 null values
In the column sweref99Ost 47.0 % of the cells have missing values


ar                              0
datum                           0
tid                             0
kommun                          0
kommunKortNamn                  0
verksamhetText                  0
sweref99Norr                    1
sweref99Ost                     1
BEJBbrandorsakText              0
arealProduktivSkogsmark_m2      0
arealAnnanTradbevuxenMark_m2    0
arealMarkUtanTrad_m2            0
TotArea                         0
Acres                           0
dtype: int64

In [140]:
sample.dtypes

ar                                       int64
datum                           datetime64[ns]
tid                                     object
kommun                                   int64
kommunKortNamn                          object
verksamhetText                          object
sweref99Norr                           float64
sweref99Ost                            float64
BEJBbrandorsakText                      object
arealProduktivSkogsmark_m2               int64
arealAnnanTradbevuxenMark_m2             int64
arealMarkUtanTrad_m2                     int64
TotArea                                  int64
Acres                                  float64
dtype: object

In [141]:
#Functions for converting easting and northing to latitudes and longitudes.
def toLat(E,N):
    lat, lon = tm.grid_to_geodetic(N,E)
    return lat
def toLon(E,N):
    lat, lon = tm.grid_to_geodetic(N,E)
    return lon

In [142]:
#Applying functions to create to new columns, Longitude and Latitude for reported fires.
sample['Latitude'] = sample.apply(lambda row: toLat(row['sweref99Ost'],row['sweref99Norr']),axis=1)
sample['Longitude'] = sample.apply(lambda row: toLon(row['sweref99Ost'],row['sweref99Norr']),axis=1)
sample

,ar,datum,tid,kommun,kommunKortNamn,verksamhetText,sweref99Norr,sweref99Ost,BEJBbrandorsakText,arealProduktivSkogsmark_m2,arealAnnanTradbevuxenMark_m2,arealMarkUtanTrad_m2,TotArea,Acres,Latitude,Longitude
86853,2018,2018-07-15,12:13:00,2361,Härjedalen,Verksamhet inte knuten till en byggnad,6850840.0,472566.0,Återantändning av brand från tidigare räddning...,35500000,0,5020000,40520000,10012.701205,61.789497,14.479805
86754,2018,2018-07-14,16:54:00,2161,Ljusdal,Verksamhet inte knuten till en byggnad,6874305.0,513847.0,Blixtnedslag,9310000,0,0,9310000,2300.549068,62.000868,15.264378
44842,2008,2008-06-10,14:03:00,2104,Hofors,Verksamhet inte knuten till en byggnad,6713643.0,566513.0,Återantändning av brand från tidigare räddning...,3000000,0,0,3000000,741.315489,60.553268,16.212820
91213,2019,2019-04-22,16:42:00,583,Motala,Verksamhet inte knuten till en byggnad,6511162.0,519856.0,Övriga gnistor,2265000,0,0,2265000,559.693194,58.740087,15.343028
86754,2018,2018-07-14,16:54:00,2161,Ljusdal,Verksamhet inte knuten till en byggnad,6874305.0,513847.0,Blixtnedslag,9310000,0,0,9310000,2300.549068,62.000868,15.264378
86891,2018,2018-07-15,14:32:00,2161,Ljusdal,Verksamhet inte knuten till en byggnad,6833095.0,525087.0,Blixtnedslag,3600000,0,0,3600000,889.578587,61.630368,15.473247
67755,2014,2014-07-31,13:31:00,1907,Surahammar,Verksamhet inte knuten till en byggnad,6635785.0,1522356.0,Övriga gnistor,95760000,14940000,17370000,128070000,31646.758227,58.630960,32.740162
86773,2018,2018-07-14,17:48:00,2303,Ragunda,Verksamhet inte knuten till en byggnad,6999852.0,570078.0,Blixtnedslag,2270000,0,0,2270000,560.928720,63.121209,16.389347
86987,2018,2018-07-16,11:32:00,2510,Jokkmokk,Verksamhet inte knuten till en byggnad,7389905.0,730955.0,Okänd,3500000,0,0,3500000,864.868071,66.541548,20.203994
44842,2008,2008-06-10,14:03:00,2104,Hofors,Verksamhet inte knuten till en byggnad,6713643.0,566513.0,Återantändning av brand från tidigare räddning...,3000000,0,0,3000000,741.315489,60.553268,16.212820


In [143]:
#We need to group fire sizes here but what sizes should we use? What is a "big" fire and what is a "small" fire?
sample['Acres'].describe()

count       50.000000
mean      5361.737248
std       9334.684194
min        518.920842
25%        772.203634
50%       1134.212698
75%       2785.492950
max      31646.758227
Name: Acres, dtype: float64

In [144]:
sample = sample[sample['Latitude'].notna()]
sample = sample[sample['Longitude'].notna()]


In [145]:
#Group fires by size
small_wildfires = folium.FeatureGroup(TotArea = '< 100 Acres')
medium_wildfires = folium.FeatureGroup(TotArea = '100 - 500 Acres')
large_wildfires = folium.FeatureGroup(TotArea = '1000 - 5000 Acres')
xl_wildfires = folium.FeatureGroup(TotArea = '> 5000 Acres')

In [146]:
#Function that takes the dataframe and returns a map. Circles on the map are colored based on the size of the fire.
def add_FireCircle(df, m):
    for i, v in df.iterrows():
        
        fire_size = float(v['Acres'])
        
        #When hoovering over the circle, the popup will show the fire name, year, cause, state and size.
        #More values can be added.
        popup = """
        Kommun : <b>%s</b><br>
        Size (Acres) : <b>%s</b><br>
        Cause : <b>%s</b><br>
        Year: <b>%s</b><br>
        """ % (v['kommunKortNamn'], v['Acres'], 
            v['BEJBbrandorsakText'], 
            v['ar'])
        
        
        if fire_size < 100:
            folium.CircleMarker(location = [v['Latitude'], 
                                            v['Longitude']],
                            radius = np.log(fire_size) * 0.8,
                            weight = 0,
                            tooltip = popup,
                            color = '#ffeda0',
                            fill_color = '#ffeda0',
                            fill_opacity = 0.7,
                            fill = True).add_to(small_wildfires)
            
        if fire_size in range(100, 500):
            folium.CircleMarker(location = [v['Latitude'], 
                                            v['Longitude']],
                            radius = np.log(fire_size),
                            weight = 0,
                            tooltip = popup,
                            color = '#feb24c',
                            fill_color = '#feb24c',
                            fill_opacity = 0.7,
                            fill = True).add_to(medium_wildfires)
        
        if fire_size in range(1000, 5000):
            folium.CircleMarker(location = [v['Latitude'], 
                                            v['Longitude']],
                            radius = np.log(fire_size) * 1.5,
                            weight = 0,
                            tooltip = popup,
                            color = '#fc4e2a',
                            fill_color = '#fc4e2a',
                            fill_opacity = 0.7,
                            fill = True).add_to(large_wildfires)

        if fire_size > 5000:
            folium.CircleMarker(location = [v['Latitude'], 
                                            v['Longitude']],
                            radius = np.log(fire_size) * 2,
                            weight = 0,
                            tooltip = popup,
                            color = '##b10026',
                            fill_color = '#b10026',
                            fill_opacity = 0.7,
                            fill = True).add_to(xl_wildfires)

    small_wildfires.add_to(m)
    medium_wildfires.add_to(m)
    large_wildfires.add_to(m)
    xl_wildfires.add_to(m)
    folium.LayerControl(collapsed = False).add_to(m)

    return m

In [147]:
#Map instanciation, with start point at the center of the US. Type of map is set to Stamen Terrain.
map = folium.Map(location = [59.334591, 18.063240],
               tiles = 'Stamen Terrain',
               zoom_start = 5.5)
map

In [148]:
m_sweden = add_FireCircle(sample, map)
m_sweden.save('sweden.html')

In [149]:
m_sweden